In [8]:
from pylab import *
import matplotlib.pyplot as plt
import os
import pyAgrum as gum

In [9]:
bn=gum.BayesNet('WaterSprinkler')
print(bn)

BN{nodes: 0, arcs: 0, domainSize: 1, dim: 0, mem: 0o}


In [10]:
id_c=bn.add(gum.LabelizedVariable('c','cloudy ?',2))
print(id_c)

0


In [11]:
id_s, id_r, id_w = [ bn.add(name, 2) for name in "srw" ] #bn.add(name, 2) === bn.add(gum.LabelizedVariable(name, name, 2))
print (id_s,id_r,id_w)
print (bn)

1 2 3
BN{nodes: 4, arcs: 0, domainSize: 16, dim: 4, mem: 64o}


In [12]:
print(f"{bn.variable(id_s)=}")
print(f"{bn['s']=}")
print(f"{id_s=}")
print(f"{bn.idFromName('s')=}")
print(f"{bn.variable(id_s).name()=}")

bn.variable(id_s)=(pyAgrum.DiscreteVariable@0000028A9C76B2E0) s:Range([0,1])
bn['s']=(pyAgrum.DiscreteVariable@0000028A9C76B2E0) s:Range([0,1])
id_s=1
bn.idFromName('s')=1
bn.variable(id_s).name()='s'


In [13]:
bn.addArc("c","s")

for link in [(id_c,id_r),(id_s,id_w),(id_r,id_w)]:
    bn.addArc(*link)
print(bn)

BN{nodes: 4, arcs: 4, domainSize: 16, dim: 9, mem: 144o}


In [14]:
import pyAgrum.lib.notebook as gnb
bn

(pyAgrum.BayesNet<double>@0000028A9C660320) BN{nodes: 4, arcs: 4, domainSize: 16, dim: 9, mem: 144o}

In [15]:
bn=gum.fastBN("c->r->w<-s<-c")
bn

(pyAgrum.BayesNet<double>@0000028A9C5BD560) BN{nodes: 4, arcs: 4, domainSize: 16, dim: 9, mem: 144o}

In [18]:
bn.cpt(id_c).fillWith([0.4,0.6]) # remember : c=  0


(pyAgrum.Potential<double>@0000028AFBF444B0) 
  c                |
0        |1        |
---------|---------|
 0.4000  | 0.6000  |

In [19]:
bn.cpt("c").fillWith([0.5,0.5])

(pyAgrum.Potential<double>@0000028AFBF444B0) 
  c                |
0        |1        |
---------|---------|
 0.5000  | 0.5000  |

In [22]:
bn.cpt("s").names

('s', 'c')

In [24]:
bn.cpt("s")[:]=[ [0.5,0.5],[0.9,0.1]]
print(bn.cpt("s")[1]) # If C=1

[0.9 0.1]


Then P(S|C=0)=[0.5,0.5] and P(S|C=1)=[0.9,0.1]

In [26]:
bn.cpt("s")[0,:]=0.5 # equivalent to [0.5,0.5]
bn.cpt("s")[1,:]=[0.9,0.1]

print(bn.cpt("w").names)
bn.cpt("w")

('w', 'r', 's')


(pyAgrum.Potential<double>@0000028AFBF44930) 
             ||  w                |
r     |s     ||0        |1        |
------|------||---------|---------|
0     |0     || 0.5418  | 0.4582  |
1     |0     || 0.4991  | 0.5009  |
0     |1     || 0.4050  | 0.5950  |
1     |1     || 0.4229  | 0.5771  |

In [27]:
bn.cpt("w")[0,0,:] = [1, 0] # r=0,s=0
bn.cpt("w")[0,1,:] = [0.1, 0.9] # r=0,s=1
bn.cpt("w")[1,0,:] = [0.1, 0.9] # r=1,s=0
bn.cpt("w")[1,1,:] = [0.01, 0.99] # r=1,s=1

In [28]:
# Probably the most convienient way

bn.cpt("w")[{'r': 0, 's': 0}] = [1, 0]
bn.cpt("w")[{'r': 0, 's': 1}] = [0.1, 0.9]
bn.cpt("w")[{'r': 1, 's': 0}] = [0.1, 0.9]
bn.cpt("w")[{'r': 1, 's': 1}] = [0.01, 0.99]
bn.cpt("w")

(pyAgrum.Potential<double>@0000028AFBF44930) 
             ||  w                |
r     |s     ||0        |1        |
------|------||---------|---------|
0     |0     || 1.0000  | 0.0000  |
1     |0     || 0.1000  | 0.9000  |
0     |1     || 0.1000  | 0.9000  |
1     |1     || 0.0100  | 0.9900  |

In [29]:
bn.cpt("r")[{'c':0}]=[0.8,0.2]
bn.cpt("r")[{'c':1}]=[0.2,0.8]

In [32]:
print(gum.availableBNExts())

bif|dsl|net|bifxml|o3prm|uai|xdsl|pkl


In [34]:
gum.saveBN(bn,"Output/WaterSprinkler.bif")
with open("Output/WaterSprinkler.bif","r") as out:
    print(out.read())

network "unnamedBN" {
// written by aGrUM 1.15.1
}

variable c {
   type discrete[2] {0, 1};
}

variable r {
   type discrete[2] {0, 1};
}

variable w {
   type discrete[2] {0, 1};
}

variable s {
   type discrete[2] {0, 1};
}

probability (c) {
   table 0.5 0.5;
}
probability (r | c) {
   (0) 0.8 0.2;
   (1) 0.2 0.8;
}
probability (w | r, s) {
   (0, 0) 1 0;
   (1, 0) 0.1 0.9;
   (0, 1) 0.1 0.9;
   (1, 1) 0.01 0.99;
}
probability (s | c) {
   (0) 0.5 0.5;
   (1) 0.9 0.1;
}


